In [2]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-07 20:21:33--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6a0dc8737cec33e9b4ff0def6f.dl-eu.dropboxusercontent.com/cd/0/inline/CyykawTozGMP4fer2iGd3d1G1ibVRz2eG5X3SnJ78v2T9pi5YWQDj2Et1n6efZ4UWR3lb2s18dB9ldl-lKjerzOFni5a0aX6xOXObeNnO63X1j3CER2wnJOKq8_dMc6eYqSNt3PwUYFZA5x6r1FfCbb6/file?dl=1# [following]
--2025-10-07 20:21:34--  https://uc6a0dc8737cec33e9b4ff0def6f.dl-eu.dropboxusercontent.com/cd/0/inline/CyykawTozGMP4fer2iGd3d1G1ibVRz2eG5X3SnJ78v2T9pi5YWQDj2Et1n6efZ4UWR3lb2s18dB9ldl-lKjerzOFni5a0aX6xOXObeNnO63X1j3CER2wnJOKq8_dMc6eYqSNt3PwUYFZA5x6r1FfCbb6/file?dl=1
Resolving uc6a0dc8737cec33e9b4ff0def6f.dl-eu.dropboxusercontent.com (uc6a0dc8737cec33e9b4ff0def6f.dl-eu.dropbo

In [3]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wikitext-filtered-full/dataset_info.json  
replace wikitext-filtered-full/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wikitext-filtered-full/state.json  
replace wikitext-filtered-full/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wikitext-filtered-full/data-00000-of-00001.arrow  y
y

Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wikitext-filtered-10k/dataset_info.json  
replace wikitext-filtered-10k/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wikitext-filtered-10k/state.json  
replace wikitext-filtered-10k/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: wikitext-filtered-10k/data-00000-of-00001.arrow  


In [4]:
!pip install datasets
import datasets

In [5]:
!pip install gensim

In [6]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


In [12]:
import nltk #Imports Natural Language Toolkit library
from nltk.corpus import stopwords #Import Stopword module for common words
from gensim.models import Word2Vec #Import Word2Vec
import string #Import String for punctuations characters

#Download stopwords
try:
    stopwords = stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    stopwords = stopwords.words('english')


def preprocess_text(text):
    tokens = text.lower().split() #Tokenize by empty space and convert to lowercase
    tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens] #Remove punctuation
    tokens = [word for word in tokens if word not in stopwords] #Remove stopwords
    return tokens

print("Preprocessing wikitext_small")
processed_small = [preprocess_text(doc['text']) for doc in wikitext_small] #Preprocess the datasets

print("Training Word2Vec model for wikitext_small") #Train Word2Vec models
model_small = Word2Vec(sentences=processed_small, vector_size=50, window=5, min_count=5, epochs=5)
#Vector size 50 = Each word will be represented by a vector of 50 numbers
#Window 5 = Model considers up to 5 words before and 5 words after the target word for context
#Min_count 5 = Filter out rare words by ignoring words that appear less than 5 times, less context to learn meaningful representation
#The model will go through the data 5 times to refine the word vectors


Preprocessing wikitext_small
Training Word2Vec model for wikitext_small


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd

# IMPORTANT: Update this path to the actual location of combined.csv in your Google Drive
file_path = '/content/drive/My Drive/wordsim353/combined.csv'

try:
    wordsim_df = pd.read_csv(file_path)
    print("WordSim-353 data loaded successfully.")
    display(wordsim_df.head())
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
    print("Please check the file path in your Google Drive and update the 'file_path' variable.")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

WordSim-353 data loaded successfully.


,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


In [ ]:
from scipy.spatial.distance import cosine
import numpy as np #Import numpy for nan

def get_word_vector(word, model): #Function to get vector for a word, handling words not in vocabulary
    try:
        return model.wv[word] #Gensim's Word2Vec model stores word vectors in the 'wv' attribute
    except KeyError:
        return None #Return None if word is not in the model's vocabulary

computed_similarities = [] #List to store computed cosine similarities

for index, row in wordsim_df.iterrows(): #Iterate through each row in the WordSim353 Data
    #Get words from the WordSim and convert to lowercase
    word1 = row['Word 1'].lower()
    word2 = row['Word 2'].lower()

    #Get vectors for the words from the trained model
    vector1 = get_word_vector(word1, model_small)
    vector2 = get_word_vector(word2, model_small)

    if vector1 is not None and vector2 is not None:
        similarity = 1 - cosine(vector1, vector2)
        #Compute cosine similarity with Cosine() from scipy.spatial.distance computes cosine distance
    else:
        #If either word is not in the model's vocabulary, we cannot compute similarity.
        #Assign NaN (Not a Number) to indicate this.
        similarity = np.nan # Use np.nan for consistency

    computed_similarities.append(similarity)

# Add the computed similarities as a new column to the DataFrame
# The column name indicates which model was used
wordsim_df['Word2Vec_Similarity_Small'] = computed_similarities

print("Cosine similarities computed using model_small.")
display(wordsim_df.head())
# Optional: Display rows where similarity could not be computed
# print("\nRows with words not in model_small vocabulary:")
# display(wordsim_df[wordsim_df['Word2Vec_Similarity_Small'].isna()])